<a href="https://colab.research.google.com/github/jwalx/Tensorflow/blob/main/04_Transfer_Learning_Part_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer learning with tensorflow part1: Feature extraction
Transfer learning is leveraging a working model's existing and learned patterns for our own problem.

There are two main benefits:
1. Can leverage an existing neural network architecture proven to work on problems similiar to our own. 
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own, then we can adapt those patterns to our own data

In [1]:
#check whether we are running on gpu
!nvidia-smi

Tue Jan 10 11:49:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#downlaod and becoming one with data
import zipfile

#download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

#unzipping
zip_ref=zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2023-01-10 11:49:52--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.128, 142.250.145.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  41.8MB/s    in 4.4s    

2023-01-10 11:49:57 (36.2 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
#check the number of dictectories
import os

# Walkthrough 10 percen data directory and list number of files
for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/c

# Creating data loaders(preparing the data)
We'll use the ImageDataGenerator class to load the data in batches

In [4]:
#setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE=(224,224)
BATCH_SIZE=32

train_dir  ="10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen=ImageDataGenerator(rescale=1/255.)
test_datagen=ImageDataGenerator(rescale=1/255.)

print("training images")
train_data_10_percent=train_datagen.flow_from_directory(train_dir,
                                                        target_size=IMAGE_SHAPE,
                                                        batch_size=BATCH_SIZE,
                                                        class_mode="categorical")
print("testing images")
test_data_10_percent=test_datagen.flow_from_directory(test_dir,
                                                      target_size=IMAGE_SHAPE,
                                                      batch_size=BATCH_SIZE,
                                                      class_mode="categorical")


training images
Found 750 images belonging to 10 classes.
testing images
Found 2500 images belonging to 10 classes.


# Setting up callbacks(things to run whilst our model trains)

Callbacks are extra functionality you can add to your models to be peformed during or after training.Some of the most popular callbacks:

* Tracking experiments with Tensorboard `callback`
* Model checkpoint with the ModelCheckpoint `callback`
* Stopping a model from training(before it trains too long and overfits) with the EarlyStopping `callback`

In [5]:
# Create a tensorboard callback(functionized because we need to create a new one for each model)
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name+"/"+ experiment_name+ "/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving Tensorboard log files to:{log_dir}")
  return tensorboard_callback

# Creating models using Tensorflow Hub

in past we've used Tensorflow to create our own model layer by layer from scratch.

Now we're going to do a similiar process,except the majority of our model's layers are going to come from Tensorflow Hub.

We can access pretrained model on TFhub

In [6]:
# Lets's compare the two models
resnet_url="https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

efficientnet_url="https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [7]:
# import dependencies 
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Creating and testing Resnet Tensorflow Hub feature Extraction model


In [8]:
IMAGE_SHAPE+(3,)

(224, 224, 3)

In [9]:
#Lets make a create model() function to create a model from a URL
def create_model(model_url,num_classes=10):
 """
 Takes a Tensorflow Hub Url an creates a Keras Sequential model with it.

 Args:
 model_url(str): A tensorflow  hub feature extraction Url.
 num_classes(int): Number of output neurons in the output layer,
                   should be equal to the number of target classess,default 10.
 
 Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor
    layer and Dense output layer with num_classes output neurons.
 """
 #download the pretrained model and save it as keras layer
 feature_extractor_layer = hub.KerasLayer(model_url,
                                          trainable=False,
                                          name="feature_extraction_layer",
                                          input_shape=IMAGE_SHAPE+(3,))
 #create our own model
 model=tf.keras.Sequential([
     feature_extractor_layer,
     layers.Dense(num_classes,activation="softmax",name="output_layer")
 ])
 return model

In [10]:
#create Resnet model
resnet_model = create_model(resnet_url,
                            num_classes = train_data_10_percent.num_classes)

In [ ]:
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer =tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])
history_resnet=resnet_model.fit(train_data_10_percent,
                                epochs=10,
                                steps_per_epoch=len(train_data_10_percent),
                                validation_data=test_data_10_percent,
                                validation_steps=len(train_data_10_percent),
                                callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                       experiment_name="resnet50V2")]
                                )

Saving Tensorboard log files to:tensorflow_hub/resnet50V2/20230110-115014
Epoch 1/10


In [ ]:
resnet_model.summary()

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns seperate loss curves for accuracy and loss
  """
  loss=history.history["loss"]
  val_loss=history.history["val_loss"]

  accuracy=history.history["accuracy"]
  val_accuracy=history.history["val_accuracy"]

  epochs=range(len(history.history["loss"]))

  # Plot loss
  plt.figure()
  plt.plot(epochs,loss,label="training_loss")
  plt.plot(epochs,val_loss,label="val_loss")
  plt.title("Loss")
  plt.xlabel("epochs")
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs,accuracy,label="accuracy")
  plt.plot(epochs,val_accuracy,label="val_accuracy")
  plt.title("Accuracy")
  plt.xlabel("epochs")
  plt.legend();


In [ ]:
plot_loss_curves(history_resnet)

## Wow!!!

That is incredible. Our transfer learning extractor model out performed all the previous models we built by hand....
and in quicker training time and wuth only 10% percent of the training data

# Creating and testing efficientnetB0 Tensorflow Hub feature Extraction model

In [ ]:
# create efficientnetB0 feature extarction layer
efficientnet_model=create_model(model_url=efficientnet_url,
                                num_classes=train_data_10_percent.num_classes)

#compile
efficientnet_model.compile(loss="categorical_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

#fit efficient net model to 10% data
history_efficientnet=efficientnet_model.fit(train_data_10_percent,
                                        epochs=5,
                                        steps_per_epoch=len(train_data_10_percent),
                                        validation_data=test_data_10_percent,
                                        validation_steps=len(test_data_10_percent),
                                        callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub/",
                                                                               experiment_name="efficientnetB0")]
                                        )

In [ ]:
plot_loss_curves(history_efficientnet)

In [ ]:
efficientnet_model.summary()

In [ ]:
resnet_model.summary()

# different types of transfer learning 

* **"As Is"** transfer learning - using existing model with no changes what so ever(e.g using ImageNet model on 1000 Imagenet classes
* **"Feature Extraction"** transfer learning- use the pretrained patterns of an existing model(eg:EfficientNETB0 trained on Imagenet) and adjust the output layer for your own problem(eg: 1000 classes > 10 classes of food)
* **"Fine tuning"** - use the prelearned patterns of an existing model and fine-tune many or all of the underlying layers(including new output layers   

# Comparing our models results with tensorboard

* note when you upload things to tensorboard.dev, your experiments are public. so if youre running private experiments(things you don't want others to see do not upload to tensorboard.dev

In [ ]:
#upload tensorboard dev records
!tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "EfficientNetB0 VS ResNet50V2" \
  --description "Comparing these two TFHub model" \
  --one_shot

Y tensorboard experiments are uploaded publically here: https://tensorboard.dev/experiment/7RnrJTQBTQeLCEw21lWDAg/